## Data loading and examination

Loading the dataset and examine its contents to gain insight into its structure 

In [27]:
# Import necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('new_york_listings_2024.csv')

# Display information about the DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20758 entries, 0 to 20757
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              20758 non-null  int64  
 1   name                            20758 non-null  object 
 2   host_id                         20758 non-null  int64  
 3   host_name                       20758 non-null  object 
 4   neighbourhood_group             20758 non-null  object 
 5   neighbourhood                   20758 non-null  object 
 6   latitude                        20758 non-null  float64
 7   longitude                       20758 non-null  float64
 8   room_type                       20758 non-null  object 
 9   price                           20758 non-null  float64
 10  minimum_nights                  20758 non-null  int64  
 11  number_of_reviews               20758 non-null  int64  
 12  last_review                     

In [28]:
# Show the first five rows of the DataFrame
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,rating,bedrooms,beds,baths
0,1312228,Rental unit in Brooklyn · ★5.0 · 1 bedroom,7130382,Walter,Brooklyn,Clinton Hill,40.683710,-73.964610,Private room,55.0,...,2015-12-20,0.03,1,0,0,No License,5.0,1,1,Not specified
1,45277537,Rental unit in New York · ★4.67 · 2 bedrooms ·...,51501835,Jeniffer,Manhattan,Hell's Kitchen,40.766610,-73.988100,Entire home/apt,144.0,...,2023-05-01,0.24,139,364,2,No License,4.67,2,1,1
2,971353993633883038,Rental unit in New York · ★4.17 · 1 bedroom · ...,528871354,Joshua,Manhattan,Chelsea,40.750764,-73.994605,Entire home/apt,187.0,...,2023-12-18,1.67,1,343,6,Exempt,4.17,1,2,1
3,3857863,Rental unit in New York · ★4.64 · 1 bedroom · ...,19902271,John And Catherine,Manhattan,Washington Heights,40.835600,-73.942500,Private room,120.0,...,2023-09-17,1.38,2,363,12,No License,4.64,1,1,1
4,40896611,Condo in New York · ★4.91 · Studio · 1 bed · 1...,61391963,Stay With Vibe,Manhattan,Murray Hill,40.751120,-73.978600,Entire home/apt,85.0,...,2023-12-03,0.24,133,335,3,No License,4.91,Studio,1,1


## Data cleaning

Removing duplicate rows and rows with missing values and irrelevant columns.

In [29]:
# Remove duplicate rows
df.drop_duplicates(inplace=True)

# Remove rows with missing values
df.dropna(inplace=True)

# Drop the irrelevant columns
df.drop(['id', 'name', 'host_id', 'host_name', 'minimum_nights', 'number_of_reviews',
         'last_review', 'reviews_per_month', 'calculated_host_listings_count',
         'availability_365', 'number_of_reviews_ltm', 'license', 'bedrooms', 'beds', 'baths',
         'room_type'],
        axis=1,
        inplace=True)

# Display the updated summary of the c
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20758 entries, 0 to 20757
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   neighbourhood_group  20758 non-null  object 
 1   neighbourhood        20758 non-null  object 
 2   latitude             20758 non-null  float64
 3   longitude            20758 non-null  float64
 4   price                20758 non-null  float64
 5   rating               20758 non-null  object 
dtypes: float64(3), object(3)
memory usage: 973.2+ KB


In [30]:
# Show the first five rows of the DataFrame
df.head()

,neighbourhood_group,neighbourhood,latitude,longitude,price,rating
0,Brooklyn,Clinton Hill,40.683710,-73.964610,55.0,5.0
1,Manhattan,Hell's Kitchen,40.766610,-73.988100,144.0,4.67
2,Manhattan,Chelsea,40.750764,-73.994605,187.0,4.17
3,Manhattan,Washington Heights,40.835600,-73.942500,120.0,4.64
4,Manhattan,Murray Hill,40.751120,-73.978600,85.0,4.91
